In [65]:
import librosa
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

In [66]:
def extract_features_mfcc(file_path, n_mfcc=13, sr=22050, n_fft=2048, hop_length=512 ):
    # Load file âm thanh
    y, sr = librosa.load(file_path, sr=sr)

    # Chuyển đổi thành MFCC
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    # Tính trung bình MFCC theo từng hệ số
    mfcc_mean = np.mean(mfcc, axis=1)
    return mfcc_mean

In [67]:
def read_folder_and_convert_to_mfcc(folder_path, label, n_mfcc=13, sr=22050, n_fft=2048, hop_length=512):
    """Đọc tất cả file .wav trong thư mục và trích xuất MFCC + nhãn"""
    X, y = [], []
    for file in tqdm(os.listdir(folder_path), desc=f"Processing {folder_path}"):
        if file.endswith('.wav'):
            file_path = os.path.join(folder_path, file)
            mfcc = extract_features_mfcc(file_path, n_mfcc=n_mfcc, sr=sr, n_fft=n_fft, hop_length=hop_length)
            X.append(mfcc)
            y.append(label)
    return X, y

In [68]:
def read_data_and_convert_to_mfcc(base_folder_path, n_mfcc=13, sr=22050, n_fft=2048, hop_length=512):
    X_train, y_train, X_test, y_test = [], [], [], []

    for folder in ['train', 'test']:
        folder_path = os.path.join(base_folder_path, folder)
        class_folders = sorted(os.listdir(folder_path))  # Lấy danh sách các lớp (sắp xếp để nhãn đúng)
        label_map = {cls: idx for idx, cls in enumerate(class_folders)}  # Gán nhãn số tự động

        for subfolder in class_folders:
            subfolder_path = os.path.join(folder_path, subfolder)
            if os.path.isdir(subfolder_path):
                label = label_map[subfolder]  # Lấy nhãn số

                # Trích xuất MFCC
                X, y = read_folder_and_convert_to_mfcc(subfolder_path, label, n_mfcc, sr, n_fft, hop_length)

                # Thêm vào tập train hoặc test
                if folder == "train":
                    X_train.extend(X)
                    y_train.extend(y)
                else:
                    X_test.extend(X)
                    y_test.extend(y)

    # Chuyển thành numpy array
    return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)

In [ ]:
if __name__ == '__main__':
    base_folder_path = 'data'
    X_train, y_train, X_test, y_test = read_data_and_convert_to_mfcc(base_folder_path)
